In [ ]:
import exVAFm as exVAF
import numpy as np
from scipy.optimize import minimize

In [ ]:
%matplotlib inline
import matplotlib.pylab as plt
plt.rcParams['font.size'] = 20
plt.rcParams['font.family'] = 'serif'
plt.rcParams['figure.figsize'] = [12,7]
plt.rcParams['axes.grid'] = True
plt.rcParams['axes.linewidth'] = 2.0
plt.rcParams['lines.linewidth'] = 2.0

In [ ]:
corh = np.array(va.getindexu("*DCH*"))
corv = np.array(va.getindexu("*DCV*"))
bpms = np.array(va.getindexu("*BPM*"))
pms  = np.array(va.getindexu("*_PM*"))

In [ ]:
# calc cost function with inputted corrector kick angles
def cor2bpm(val,corh,corv):
    global sola,solx, ncount
    ncount +=1
    
    # set corrector parameters
    for i,(j,k) in enumerate(zip(corh,corv)):
        va.setelem(j,'theta_x',float(val[0::2][i]))
        va.setelem(k,'theta_y',float(val[1::2][i]))

    # run simulation
    va.tcs()
    
    xbpms = va.LD[bpms,1]
    ybpms = va.LD[bpms,2]
    r2bpms = np.sqrt(xbpms*xbpms+ybpms*ybpms)
    
    # calculate cost function
    cost = np.amax(r2bpms)*np.average(r2bpms)*np.std(r2bpms)
    
    # store best input values
    if cost<sola :
        sola = cost
        solx = val
    return(cost)

# reset corrector parameters
def cor2reset():
    for i,(j,k) in enumerate(zip(corh,corv)):
        va.setelem(j,'theta_x',0.0)
        va.setelem(k,'theta_y',0.0)
    va.tcs()

In [ ]:
cor2reset()
plt.plot(va.LD[0:,0],va.LD[0:,1], label='$x_b$')
plt.plot(va.LD[0:,0],va.LD[0:,2], label='$y_b$')
plt.legend(loc='best')
plt.ylabel('orbit [mm]')
plt.xlabel('$z$ [m]')
plt.title('Default Orbit')
plt.show()

In [ ]:
# Set initial parameter for orbit correction
var = len(corh)+len(corv)#90 # number of correctors both x and y
x0 = np.zeros(var) # initial corrector kick angles (=0.0)
bnds = np.array([(-3e-3, 3e-3) for _ in range(len(x0))],dtype=float)

In [ ]:
# Run minimization
sola = 1e256 # worst cost value
ncount = 0   # check number of evaluations
res=minimize(cor2bpm,x0,args=(corh,corv,),method='L-BFGS-B',\
             bounds=bnds,\
             options={'maxiter':10, 'disp':False, 'eps':1.0e-7})

In [ ]:
cor2bpm(solx,corh,corv)
plt.plot(va.LD[:,0],va.LD[:,1], label='$x$')
plt.plot(va.LD[:,0],va.LD[:,2], label='$y$')

cor2reset()
plt.plot(va.LD[0:,0],va.LD[0:,1], '--',label='$x_b$')
plt.plot(va.LD[0:,0],va.LD[0:,2], '--',label='$y_b$')

plt.legend(loc='best')
plt.ylabel('orbit [mm]')
plt.xlabel('$z$ [m]')
plt.title('Corrected Orbit')
plt.show()

In [ ]:
print va.__doc__

In [ ]:
s1,d1 = va.tcs_seg(0,409)
plt.plot(d1[:,0],d1[:,1])
sd = va.SaveBeam(s1)

va.setelem(410,'B',5.0)
s2,d2 = va.tcs_seg(409,822,SD=sd)
plt.plot(d2[:,0],d2[:,1],label='$B_z$ = 5.0')

va.setelem(410,'B',10.0)
s3,d3 = va.tcs_seg(409,822,SD=sd)
plt.plot(d3[:,0],d3[:,1],label='$B_z$ = 10.0')

va.setelem(410,'B',15.0)
s4,d4 = va.tcs_seg(409,822,SD=sd)
plt.plot(d4[:,0],d4[:,1],label='$B_z$ = 15.0')

plt.legend(loc='best')
plt.ylabel('orbit [mm]')
plt.xlabel('$z$ [m]')
plt.title('Case study: SOL3_D1594')

plt.show()